This file should only be used to contain the code used to download data by using the selenium 

and

Upload data to minio

In [3]:
# %pip install boto3
# Libraries maybe you need to install in your environment to use selenium and chromedriver below
%pip install selenium chromedriver-binary

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for chromedriver-binary: filename=chromedriver_binary-125.0.6377.0.0-py3-none-any.whl size=8615952 sha256=22126749e5da1632fe326bfca7f2f5611fd9af1e349769d291e07191fe6803ff
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\5a\a9\14\64667cfea0ad833d7f5e0b72febcdf9db57032cc47c7ff9549
Successfully built chromedriver-binary


In [39]:
import os
import boto3
from selenium import webdriver
# import chromedriver_binary
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from zipfile import ZipFile

In [34]:
# The file download processing section should be placed here

def dowload_file_from_driver(path):
    # Install and initialize ChromeDriver using ChromeDriverManager
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(
        ChromeDriverManager().install()), options=chrome_options)

    #Open link
    driver.get(path)
    ignored_exceptions = (NoSuchElementException,
                        StaleElementReferenceException)

    # Wait until the file list is clickable
    file = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id=":1"]/div/c-wiz/div/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/c-wiz/div/div')))

    #Move to file
    action = ActionChains(driver)
    action.move_to_element(file)

    # Wait until the download button is clickable
    btn = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id=":1"]/div/c-wiz/div/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/c-wiz/div/div/div/div[6]')))
    btn.click()

    time.sleep(10)
    driver.quit()
#...........................................................

In [35]:
dowload_file_from_driver("https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC?usp=sharing")

In [41]:
#unzip to folder Test
with ZipFile("C:\\Users\\admin\\Downloads\\data.zip", 'r') as zip_ref:
    zip_ref.extractall(path="D:\\Work\\Test")

In [45]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'PmQYxTR8Ls9xlFUloH1H')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', 'XPuYiU74d12m05k5u6BgYHbGibKRTNR1tmdss3yh')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')


In [43]:
# Function to upload file to minio
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)


In [46]:
# Upload file to minio
upload_file_to_minio('data.json', 'data', 'data-raw/data.json')
upload_file_to_minio('data2.json', 'data', 'data-raw/data2.json')
upload_file_to_minio('data3.json', 'data', 'data-raw/data3.json')